# Crear la clase  Target

## Ejercicio 1

Cree el archivo competencia_01.csv, usando el competencia_01_crudo.csv con una variable adicional llamada clase_ternaria, que contenga las categorías **CONTINUA, BAJA+1 y BAJA+2**

Ayudita
Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**.

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction) Procedemos a instalarla, esto se debe ejecutar una sola vez

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb**

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

Y ya podemos usar **SQL** dentro de una notebook!

In [ ]:
%%sql
select 'hola mundo'

Para cargar el archivo `.csv` a una tabla pasando su ubicación a través de la variable `dataset_path`:

In [ ]:
dataset_path = '/home/aleb/DMEyF/2024/datos/'
dataset_file = 'competencia_01_crudo.csv'

In [ ]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [ ]:
%sql select * from competencia_01_crudo limit 5

In [12]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_01_crudo
group by foto_mes

,foto_mes,cantidad
0,202101,161527
1,202102,162155
2,202103,163109
3,202104,163418
4,202105,163917
5,202106,164313


In [14]:
%%sql
SELECT numero_de_cliente
FROM competencia_01_crudo
GROUP BY numero_de_cliente
HAVING COUNT(DISTINCT foto_mes) = 2
   AND MIN(foto_mes) = 202101
   AND MAX(foto_mes) = 202102
LIMIT 10;


,numero_de_cliente
0,922093456
1,922650178
2,925803620
3,927660124
4,930544172
5,931180023
6,934289853
7,934665866
8,934962459
9,939552780


In [16]:
%%sql
SELECT numero_de_cliente
FROM competencia_01_crudo
GROUP BY numero_de_cliente
HAVING COUNT(DISTINCT foto_mes) = 6
   AND MIN(foto_mes) = 202101
   AND MAX(foto_mes) = 202106
LIMIT 10;


,numero_de_cliente
0,918123743
1,918123965
2,918147740
3,918152479
4,918168525
5,918170566
6,918178517
7,918179533
8,918183034
9,918185810


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide.

In [ ]:
%%sql
create or replace table competencia_01 as
...

In [ ]:
%sql select * from competencia_01 limit 5

## Ejercicio 1.1
* ¿Cuál es la nominalidad de cada clase?


In [ ]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

* ¿Cuál es la proporción del target?

## Guardar tabla en .csv

Para guardar a un `.csv` simplemente debe ejecutar la siguiente sentencia

In [ ]:
%%sql COPY competencia_01 TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)